In [1]:
#Imports
import h5py
import pandas as pd
import numpy as np
import seaborn as sns
import random
from keras.models import load_model
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score 
import itertools
from sklearn import metrics
from matplotlib import pyplot as plt

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

import tensorflow as tf
from tensorflow.keras.optimizers import Adam


In [2]:
# load data
X = np.load('Xdata.npy')
Y = np.load('Ydata.npy')

In [3]:
#Split data
X_train, X_rem, Y_train, Y_rem = train_test_split(X, Y, test_size = 0.4, random_state=5)
X_test, X_val, Y_test, Y_val = train_test_split(X_rem, Y_rem, test_size = 0.5, random_state=5)

In [4]:
# VGG16 MODEL
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense

vgg16_model = VGG16(include_top=False,
    weights="imagenet",
    input_shape=(100,100,3))

for layer in vgg16_model.layers:
    layer.trainable = False

x = vgg16_model.output
x = Dense(256, activation = "relu")(x)
x = Dropout(.2)(x)
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=vgg16_model.input, outputs=predictions)

In [5]:
#model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [6]:
#model specs 
epochs = 30
batch_size = 100
early_stopping = EarlyStopping(monitor='accuracy',
                                patience=2,
                                verbose=1 ,
                                min_delta=0.005,
                                mode='max')
init_lr = 1e-3   #learning rate for the optimizer
optimizer = Adam(learning_rate = init_lr)

In [7]:
#model compiling
model.compile(optimizer = optimizer , loss = "BinaryCrossentropy", metrics=["accuracy" , 'AUC', 'Precision', 'Recall'])

In [8]:
print("Num GPUs Available: ", len(tf.config.list_logical_devices('GPU')))

Num GPUs Available:  2


In [ ]:
#model fitting
with tf.device("/device:GPU:0"):
    history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
              validation_data = (X_val, Y_val), verbose = 1, callbacks=[early_stopping])

Epoch 1/30
303/303 [==============================] - 158s 517ms/step - loss: 0.5680 - accuracy: 0.7935 - auc: 0.8424 - precision: 0.8177 - recall: 0.8417 - val_loss: 0.4203 - val_accuracy: 0.8283 - val_auc: 0.8973 - val_precision: 0.8398 - val_recall: 0.8716
Epoch 2/30
303/303 [==============================] - 171s 565ms/step - loss: 0.3523 - accuracy: 0.8572 - auc: 0.9253 - precision: 0.8683 - recall: 0.8968 - val_loss: 0.4038 - val_accuracy: 0.8450 - val_auc: 0.9099 - val_precision: 0.8726 - val_recall: 0.8595
Epoch 3/30
303/303 [==============================] - 169s 559ms/step - loss: 0.3023 - accuracy: 0.8776 - auc: 0.9437 - precision: 0.8896 - recall: 0.9077 - val_loss: 0.3987 - val_accuracy: 0.8420 - val_auc: 0.9120 - val_precision: 0.8551 - val_recall: 0.8774
Epoch 4/30
173/303 [================>.............] - ETA: 58s - loss: 0.2617 - accuracy: 0.8960 - auc: 0.9577 - precision: 0.9060 - recall: 0.9209

In [ ]:
sns.lineplot(data=history['accuracy'])

In [ ]:
plt.imshow(X[9800])
plt.show()

In [ ]:
model.predict(X[9800].reshape(-1,100,100,3))


In [ ]:
#evaluate model:
result = model.evaluate(X_test, Y_test, verbose=1)
# print('The loss equal to' , result[0])
# print('The accuarcy is' , result[1])

In [ ]:
# Saving model
model.save('modelvgg16-92.h5')

In [ ]:
model = load_model('model.h5')

In [ ]:
y_pred =  model.predict(X_test)
y_pred = np.argmax(y_pred,axis = 1)
y_true = np.argmax(Y_test,axis = 1) 
metrics.ConfusionMatrixDisplay.from_predictions(y_true , y_pred)